In [1]:
import itertools
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Data collecting and processing

In [2]:
df = pd.read_csv("movies.csv")

In [3]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
print(df.shape)

(4803, 24)


In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [6]:
print(df.describe())

             index        budget             id   popularity       revenue  \
count  4803.000000  4.803000e+03    4803.000000  4803.000000  4.803000e+03   
mean   2401.000000  2.904504e+07   57165.484281    21.492301  8.226064e+07   
std    1386.651002  4.072239e+07   88694.614033    31.816650  1.628571e+08   
min       0.000000  0.000000e+00       5.000000     0.000000  0.000000e+00   
25%    1200.500000  7.900000e+05    9014.500000     4.668070  0.000000e+00   
50%    2401.000000  1.500000e+07   14629.000000    12.921594  1.917000e+07   
75%    3601.500000  4.000000e+07   58610.500000    28.313505  9.291719e+07   
max    4802.000000  3.800000e+08  459488.000000   875.581305  2.787965e+09   

           runtime  vote_average    vote_count  
count  4801.000000   4803.000000   4803.000000  
mean    106.875859      6.092172    690.217989  
std      22.611935      1.194612   1234.585891  
min       0.000000      0.000000      0.000000  
25%      94.000000      5.600000     54.000000  
50%

In [7]:
print(df.isnull().sum())

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64


In [8]:
print(df.duplicated().sum())

0


### Main Features
- genres
- keywords
- original_title
- overview
- popularity
- tagline
- title
- cast
- director
- vote_average

In [9]:
print(df.columns)

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')


In [10]:
features = ["genres","keywords","overview","tagline","title","cast","director"]

In [11]:
for feature in features:
    df[feature] = df[feature].fillna("")

In [12]:
combined_features = df["genres"]+" "+df["keywords"]+" "+df["overview"]+" "+df["tagline"]+" "+df["title"]+" "+df["cast"]+" "+df["director"]

In [13]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      When ambitious New York attorney Sam is sent...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


## Building Algorithm

In [14]:
Vectorizator = TfidfVectorizer()

In [15]:
vector_data = Vectorizator.fit_transform(combined_features)

In [16]:
vector_data.shape

(4803, 31349)

In [17]:
print(vector_data)

  (0, 4405)	0.13007397510161756
  (0, 14533)	0.08286987957510253
  (0, 23687)	0.1475450502196134
  (0, 18206)	0.12094554394544946
  (0, 15951)	0.16869852873908397
  (0, 26493)	0.11410401853770415
  (0, 30347)	0.14830419354529692
  (0, 25397)	0.1564513257868596
  (0, 24138)	0.16540428104909333
  (0, 31299)	0.14830419354529692
  (0, 30893)	0.17959863615914348
  (0, 24180)	0.10875369262389524
  (0, 2127)	0.1996044560566861
  (0, 19876)	0.030846726810594093
  (0, 30868)	0.0710516516949864
  (0, 9290)	0.14044787266458833
  (0, 5389)	0.16249854865534707
  (0, 1037)	0.12044205276704567
  (0, 1275)	0.0517167846401998
  (0, 22040)	0.16390865881038189
  (0, 1315)	0.031279256768719746
  (0, 20121)	0.15869561879031263
  (0, 10623)	0.13632661307394978
  (0, 3061)	0.09808742782138465
  (0, 28189)	0.14279919495279256
  :	:
  (4802, 19703)	0.058390572523091544
  (4802, 12241)	0.07492190909958076
  (4802, 30413)	0.07402585257774721
  (4802, 9576)	0.0674085486675589
  (4802, 19949)	0.05488577822704526
 

In [18]:
similarity_score = cosine_similarity(vector_data)

In [19]:
print(similarity_score)

[[1.         0.05399851 0.03084844 ... 0.02434614 0.02809646 0.00665587]
 [0.05399851 1.         0.04098103 ... 0.05353934 0.03667414 0.01552356]
 [0.03084844 0.04098103 1.         ... 0.02269726 0.0425641  0.01228775]
 ...
 [0.02434614 0.05353934 0.02269726 ... 1.         0.02986951 0.03920937]
 [0.02809646 0.03667414 0.0425641  ... 0.02986951 1.         0.02879619]
 [0.00665587 0.01552356 0.01228775 ... 0.03920937 0.02879619 1.        ]]


In [20]:
similarity_score.shape

(4803, 4803)

In [21]:
search_name = input('Enter your movie name: ')
print(search_name)

batman


In [22]:
titles_list = df["title"].tolist()

In [23]:
print(titles_list)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [24]:
close_matches = difflib.get_close_matches(search_name, titles_list)

In [25]:
print(close_matches)

['Batman', 'Batman', 'Catwoman']


In [26]:
index_search_output = df[df.title == close_matches[0]]

In [27]:
index_search_output

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
1359,1359,35000000,Fantasy Action,NaN,268,double life dc comics dual identity chemical c...,en,Batman,The Dark Knight of Gotham City begins his war ...,44.104469,...,126.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Have you ever danced with the devil in the pal...,Batman,7.0,2096,Jack Nicholson Michael Keaton Kim Basinger Mic...,"[{'name': 'Ray Lovejoy', 'gender': 2, 'departm...",Tim Burton
4267,4267,1377800,Family Adventure Comedy Science Fiction Crime,NaN,2661,submarine dc comics missile shark attack rescue,en,Batman,The Dynamic Duo faces four super-villains who ...,9.815394,...,105.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,He's Here Big As Life In A Real Bat-Epic,Batman,6.1,203,Adam West Burt Ward Lee Meriwether Cesar Romer...,"[{'name': 'Bob Kane', 'gender': 2, 'department...",Leslie H. Martinson


In [28]:
output = itertools.chain(*similarity_score[index_search_output["index"]])

In [29]:
similarity = list(enumerate(output))

In [30]:
print(similarity)

[(0, 0.029562008213074295), (1, 0.05417408955309168), (2, 0.023571559384785758), (3, 0.2867041425184533), (4, 0.03378492835305784), (5, 0.06011430390137605), (6, 0.024477423855073052), (7, 0.029097287813273004), (8, 0.06834913087354055), (9, 0.1619098700446821), (10, 0.07339493001498477), (11, 0.03002435576992518), (12, 0.04741819103812327), (13, 0.021260706484200323), (14, 0.08553897995765518), (15, 0.04366662288324387), (16, 0.04599789673211068), (17, 0.055897318523054046), (18, 0.050949537175099965), (19, 0.048541968185221836), (20, 0.05204436366047989), (21, 0.04727888326024395), (22, 0.029982281043800102), (23, 0.01974005343196452), (24, 0.032021961589345796), (25, 0.03315473246083815), (26, 0.03557487443381056), (27, 0.03710641759244948), (28, 0.008484157095294476), (29, 0.02599382891317802), (30, 0.10676233851141839), (31, 0.022722165346343394), (32, 0.05155108388839143), (33, 0.0146468232932761), (34, 0.0073474887609145), (35, 0.04286964701966557), (36, 0.0649285817783442), (37

In [31]:
sorted_similar_movies = sorted(similarity, key = lambda x:x[1], reverse = True)

In [32]:
print(sorted_similar_movies)

[(1359, 1.0000000000000002), (9070, 1.0000000000000002), (428, 0.33059656208307275), (119, 0.2883065147705112), (3, 0.2867041425184533), (210, 0.268001372252467), (65, 0.2634096629492594), (3854, 0.25353881864373023), (299, 0.21808608978979957), (5231, 0.17185746727652468), (9, 0.1619098700446821), (5013, 0.14286816233768596), (5102, 0.14244474622550776), (4922, 0.13728123773620107), (4868, 0.13283438917662907), (8657, 0.1302813959139678), (4817, 0.12446173507368084), (8231, 0.12193166680455539), (4875, 0.11697465699980768), (5220, 0.11627007715576303), (5616, 0.11557461756551655), (8961, 0.11149852323595662), (6806, 0.11086563241475529), (9136, 0.11086393438250051), (4812, 0.10999117454834666), (6714, 0.10973067426688823), (4806, 0.10856725753990377), (30, 0.10676233851141839), (72, 0.10585618654524061), (1469, 0.10259321340270487), (4267, 0.1018558132518317), (6162, 0.1018558132518317), (2291, 0.0998756069627733), (1390, 0.09922277849423305), (1512, 0.0958057990234888), (2029, 0.0953

In [33]:
df.set_index("index")

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
index,,,,,,,,,,,,,,,,,,,,,
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]",...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,9000,Comedy Romance,NaN,72766,,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictur

In [34]:
# i = 1
# print("Movie recommendations: \n")
# for movie in sorted_similar_movies:
#     index = movie[0]
#     recommendation = df[df["index"] == index]["title"].tolist()
#     if i <= 10:
#         print(i, "." ,recommendation[0])
#     else: break
#     i+=1

## Complete Movie Recommendation System

In [35]:

search_name = input('Enter your movie name: ')

titles_list = df["title"].tolist()

close_matches = difflib.get_close_matches(search_name, titles_list)

index_search_output = df[df.title == close_matches[0]]

output = itertools.chain(*similarity_score[index_search_output["index"]])

similarity = list(enumerate(output))

sorted_similar_movies = sorted(similarity, key = lambda x:x[1], reverse = True)

df.set_index("index")

i = 1
print("Movie recommendations: \n")
for movie in sorted_similar_movies:
    index = movie[0]
    recommendation = df[df["index"] == index]["title"].tolist()
    if i <= 10:
        if(len(recommendation) != 0):
            print(i, "." ,recommendation[0])
            i+=1
    else: break
    

Movie recommendations: 

1 . Batman
2 . Batman Returns
3 . Batman Begins
4 . The Dark Knight Rises
5 . Batman & Robin
6 . The Dark Knight
7 . Batman: The Dark Knight Returns, Part 2
8 . Batman Forever
9 . Batman v Superman: Dawn of Justice
10 . Spider-Man 2
